In [1]:
import torch
from torch.nn import TransformerEncoder, TransformerEncoderLayer

In [2]:
d_model, d_hidden, n_layers, n_head = 9, 25, 2, 3

In [3]:
transformer = TransformerEncoder(
    TransformerEncoderLayer(
        d_model=d_model, 
        nhead=n_head, 
        dim_feedforward=d_hidden
    ), 
    num_layers=n_layers,
)


/anaconda/envs/torch23/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [4]:
x = torch.rand(10, 32, d_model)

In [5]:

transformer(x)

tensor([[[-1.5476,  1.8302,  1.3957,  ..., -0.8432, -0.7335, -0.2168],
         [ 1.0737, -1.7362,  0.7635,  ...,  0.1460, -0.4618, -0.2131],
         [-0.9800,  1.0716, -0.1087,  ..., -0.0137, -1.6260, -0.7862],
         ...,
         [-1.0890, -0.6231,  0.0078,  ...,  0.6666,  1.6985, -1.2145],
         [-0.5572,  1.0486,  1.3972,  ...,  0.7903, -1.1833, -0.8511],
         [ 0.7919,  0.2556,  0.0928,  ..., -1.0410,  0.2561, -0.6905]],

        [[ 1.7895, -0.4732,  0.8699,  ...,  0.4809,  0.5714, -1.8472],
         [ 1.8363, -1.6493, -0.0055,  ..., -0.0868, -0.0857, -0.5371],
         [ 1.6409,  0.7746, -1.4141,  ...,  0.6179, -0.0490, -1.0334],
         ...,
         [ 1.5844, -0.3869,  0.2327,  ...,  0.5072,  1.0003, -1.0640],
         [ 1.4336, -1.8003,  0.3263,  ...,  1.3491, -0.7787, -0.8774],
         [ 1.2479, -1.4072, -0.0727,  ...,  0.9346,  1.4538, -0.2282]],

        [[ 0.8145,  0.4068,  2.0842,  ...,  0.2739, -0.4320, -0.5553],
         [ 1.2468, -0.5254,  0.5174,  ..., -0

In [7]:

transformer(x, is_causal = True)


RuntimeError: Need attn_mask if specifying the is_causal hint. You may use the Transformer module method `generate_square_subsequent_mask` to create this mask.

In [33]:
torch.triu(torch.ones(1, 3), diagonal=1)

tensor([[0., 1., 1.]])

In [23]:
import torch
from torch.nn import TransformerEncoder, TransformerEncoderLayer

# Define the sequence length and model dimensions
seq_len = 4
d_model = 8
nhead = 2

# Create a sample input sequence (sequence length = 4, batch size = 1, feature size = 8)
input_sequence = torch.rand((seq_len, 1, d_model))

# Create a causal mask (lower triangular matrix)
causal_mask1 = torch.tril(torch.ones((seq_len, seq_len))).to(torch.bool)
causal_mask2 = torch.nn.Transformer.generate_square_subsequent_mask(seq_len).to(torch.bool)

# Define the Transformer encoder layer and encoder
encoder_layer = TransformerEncoderLayer(d_model=d_model, nhead=nhead)
transformer_encoder = TransformerEncoder(encoder_layer, num_layers=1)
transformer_encoder.eval()

# Apply the Transformer encoder with the attention mask
output1 = transformer_encoder(input_sequence, mask=causal_mask1, need_weights = True)
print(output1)
output2 = transformer_encoder(input_sequence, mask=causal_mask2, is_causal=True)

# print("Input Sequence:\n", input_sequence)
# print("Attention Mask:\n", causal_mask)
print("Output Sequence I:\n", output1)
print("Output Sequence II:\n", output2)
print(torch.allclose(output1, output2, atol=1e-6))    

/anaconda/envs/torch23/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


TypeError: TransformerEncoder.forward() got an unexpected keyword argument 'need_weights'

In [10]:
causal_mask1

tensor([[ True, False, False, False],
        [ True,  True, False, False],
        [ True,  True,  True, False],
        [ True,  True,  True,  True]])

In [11]:
causal_mask2

tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])

In [34]:
import numpy as np
size = 4
inputs = torch.ones((4,3))
maten = torch.nn.MultiheadAttention(embed_dim=3, num_heads=1)

In [37]:
maten.eval()
arr = [[-np.inf for _ in range(size)] for _ in range(size)]
arr = torch.tensor(arr)
attn_mask1 = torch.triu(arr, diagonal=1) # Here diagonal=1 is used to make diagonal entries 0

attn_mask2 = torch.nn.Transformer.generate_square_subsequent_mask(size).to(torch.bool)
query2 = maten(inputs, inputs, inputs, attn_mask=attn_mask1, need_weights = True)
query2

(tensor([[ 0.0799, -0.1029,  0.0617],
         [ 0.0799, -0.1029,  0.0617],
         [ 0.0799, -0.1029,  0.0617],
         [ 0.0799, -0.1029,  0.0617]], grad_fn=<SqueezeBackward1>),
 tensor([[1.0000, 0.0000, 0.0000, 0.0000],
         [0.5000, 0.5000, 0.0000, 0.0000],
         [0.3333, 0.3333, 0.3333, 0.0000],
         [0.2500, 0.2500, 0.2500, 0.2500]], grad_fn=<SqueezeBackward1>))

In [39]:
attn_mask1

tensor([[0., -inf, -inf, -inf],
        [0., 0., -inf, -inf],
        [0., 0., 0., -inf],
        [0., 0., 0., 0.]])

In [38]:
attn_mask2

tensor([[False,  True,  True,  True],
        [False, False,  True,  True],
        [False, False, False,  True],
        [False, False, False, False]])

In [29]:
query2 = maten(inputs, inputs, inputs, attn_mask=attn_mask, need_weights = False)
query2

(tensor([[-0.0353,  0.3570, -0.6883],
         [-0.0353,  0.3570, -0.6883],
         [-0.0353,  0.3570, -0.6883],
         [-0.0353,  0.3570, -0.6883]], grad_fn=<SqueezeBackward1>),
 None)

In [32]:
import torch 
import torch.nn as nn

batch_size = 4
seq_len = 3
embedding_dim = 8
num_heads = 2

mha=nn.MultiheadAttention(num_heads=num_heads, embed_dim=embedding_dim, batch_first=True)
x = torch.randn(batch_size, seq_len, embedding_dim)

mask = nn.Transformer.generate_square_subsequent_mask(seq_len)

with torch.backends.cuda.sdp_kernel(enable_math=True, enable_mem_efficient=False, enable_flash=False):
    no_mask = mha(x,x,x, need_weights=False)[0]
    with_attn_mask = mha(x,x,x, need_weights=True, attn_mask=mask)[0]
    with_is_causal_need_weights = mha(x,x,x, need_weights=True, is_causal=True, attn_mask=mask)[0]
    with_is_causal_no_need_weights = mha(x,x,x, need_weights=False, is_causal=True, attn_mask=mask)[0]
    
#succeeds    
assert with_attn_mask.allclose(with_is_causal_no_need_weights)

#both should succeed but fail
assert with_attn_mask.allclose(with_is_causal_need_weights), "is_causal should match regardless of 'need_weights'"
assert not no_mask.allclose(with_is_causal_need_weights),  "no mask should NOT match is_causal=True, need_weights=True"
    